In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys; sys.path.insert(0, '..')

In [3]:
import pandas as pd
import numpy as np
import geopandas as gpd
from qr_code import QR_CODE, get_compact

In [4]:
def print_grid(arr):
    black_square = "█"       # U+2B1B - BLACK LARGE SQUARE
    white_square = " "       # U+2B1C - WHITE LARGE SQUARE
    return [''.join(np.where(row, black_square, white_square)) for row in arr]

## Load

In [35]:
# gdf = gpd.read_file('data/150m_grid.geojson')
gdf = gpd.read_file('../data/usng_clipped.geojson')

In [36]:
gdf["fill"] = False
gdf["isqr"] = False

In [54]:
df = pd.DataFrame(gdf.drop('geometry', axis=1))

### Make Square

In [55]:
df['easting'] = gdf['USNGCoord'].str.split().str[-2].astype('int')
df['northing'] = gdf['USNGCoord'].str.split().str[-1].astype('int')

In [56]:
df['easting'].sort_values().unique()

array([220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232,
       233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245,
       246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258,
       259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271,
       272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284,
       285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297,
       298, 299, 300, 301, 302])

In [57]:
df[df['easting'] >= 222]

,fid,GZD,SQID,USNGCoord,Place,Region,fill,isqr,easting,northing
0,9727,19T,CG,19T CG 222 913,Boston MA,Eastern,False,False,222,913
1,9726,19T,CG,19T CG 222 911,Boston MA,Eastern,False,False,222,911
2,9736,19T,CG,19T CG 223 913,Boston MA,Eastern,False,False,223,913
3,1575,19T,CG,19T CG 223 912,Boston MA,Eastern,False,False,223,912
4,9744,19T,CG,19T CG 224 913,Boston MA,Eastern,False,False,224,913
...,...,...,...,...,...,...,...,...,...,...
5955,3746,19T,CG,19T CG 297 957,Boston MA,Eastern,False,False,297,957
5956,3754,19T,CG,19T CG 298 957,Boston MA,Eastern,False,False,298,957
5957,19246,19T,CG,19T CG 295 956,Boston MA,Eastern,False,False,295,956
5958,19253,19T,CG,19T CG 296 956,Boston MA,Eastern,False,False,296,956


## Reshape grid

In [8]:
gdf.shape

(5960, 9)

In [9]:
w, h = 53, 49

In [10]:
fill_array = gdf['fill'].values.reshape(w, h).T
isqr_array = gdf['isqr'].values.reshape(w, h).T

ValueError: cannot reshape array of size 5960 into shape (53,49)

## Place QR code inside grid

In [10]:
qr_arr = get_compact(QR_CODE)
qr_arr.shape

(25, 25)

In [11]:
# make array same size as qr_arr but all True
qr_size_arr = np.ones_like(qr_arr, dtype=bool)

In [12]:
start_x, start_y = 10, 10

In [13]:
# add qr code to fill_array
start_row, start_col = start_x, start_y  # example coordinates
fill_array[start_row:start_row+25, start_col:start_col+25] = qr_arr
fill_array[start_row:start_row+25, start_col:start_col+25] = qr_arr

In [14]:
print_grid(fill_array)

['                                                     ',
 '                                                     ',
 '                                                     ',
 '                                                     ',
 '                                                     ',
 '                                                     ',
 '                                                     ',
 '                                                     ',
 '                                                     ',
 '                                                     ',
 '          ███████   ██ ███  ███████                  ',
 '          █     █ █████  █  █     █                  ',
 '          █ ███ █ █  █   ██ █ ███ █                  ',
 '          █ ███ █   █   ██  █ ███ █                  ',
 '          █ ███ █ █ ██ ████ █ ███ █                  ',
 '          █     █ ██   █ ██ █     █                  ',
 '          ███████ █ █ █ █ █ ███████                  ',
 '            

In [15]:
# # add qr code to fill_array
# start_row, start_col = 30, 40  # example coordinates
# fill_array[start_row:start_row+25, start_col:start_col+25] = qr_arr
# fill_array[start_row:start_row+25, start_col:start_col+25] = qr_arr

In [16]:
# add qr_size_arr to isqr_array
start_row, start_col = start_x, start_y  # example coordinates
isqr_array[start_row:start_row+25, start_col:start_col+25] = qr_size_arr
isqr_array[start_row:start_row+25, start_col:start_col+25] = qr_size_arr

In [18]:
# print_grid(fill_array)

In [19]:
# print_grid(isqr_array)

## Un-reshape

In [20]:
gdf['fill'] = fill_array.T.flatten()
gdf['isqr'] = isqr_array.T.flatten()

In [21]:
gdf.to_file('out/output.geojson', driver='GeoJSON')